# BioRAG-A-Retrieval-Augmented-Agent-for-Scientific-Literature-Analysis

## Extraction and Initial Document Creation - Indexing

In [1]:
!nvidia-smi

Sat Jan 17 19:13:39 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:CA:00.0 Off |                    0 |
| N/A   30C    P0             45W /  300W |       4MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip uninstall -y numpy ml_dtypes onnx onnxruntime
!pip install numpy==1.26.4
!pip install ml_dtypes==0.2.0
!pip install onnx==1.14.1
!pip install onnxruntime==1.16.3
!pip uninstall -y scipy
!pip install scipy==1.10.1
!pip install 'pdf2image'
!pip install unstructured_inference
!pip install langchain[community]
!pip install "unstructured[all]"
!pip install pi-heif
!pip install unstructured-pytesseract

Found existing installation: numpy 1.26.3
Uninstalling numpy-1.26.3:
  Successfully uninstalled numpy-1.26.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 105.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 38.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 17.7 MB/s eta 0:00:00
Found existing installation: scipy 1.12.0
Uninstalling scipy-1.12.0:
  Successfully uninstalled scipy-1.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 43.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━

In [1]:
## Loading a document
from langchain_community.document_loaders import UnstructuredPDFLoader
import os
os.environ["TESSDATA_PREFIX"] = "/opt/conda/share/tessdata"
file_name = "/home/jovyan/work/BioRAG/Data/pdfs/cells-11-02650-v2.pdf"
loader = UnstructuredPDFLoader(file_name,
                               languages = ['eng'],
                               mode = "elements",
                               strategy = "hi_res",
                               infer_table_structure= True,
                              )
docs = loader.load()

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [2]:
cats = set()
for do in docs:
    cats.add(do.metadata.get('category'))
print(cats)

{'NarrativeText', 'Header', 'Title', 'ListItem', 'Image', 'Table', 'FigureCaption', 'UncategorizedText', 'Formula'}


### Key Notes:
So, keeping only these four categories: Title, NarrativeText, Table, FigureCaption

In [3]:
Docs = docs.copy()
Docs = [doc for doc in Docs if doc.metadata.get('category') not in ['Image', 'Header', 'ListItem', 'UncategorizedText']]
print(f"Number of filtered docs: {len(Docs)}")

Number of filtered docs: 140


In [4]:
cats = set()
for do in Docs:
    cats.add(do.metadata.get('category'))
print(cats)

{'NarrativeText', 'Title', 'FigureCaption', 'Table', 'Formula'}


In [5]:
# serach for unnecessary narrative texts like 18 of 22 etc
s= set()
for do in Docs:
    if do.metadata.get('category') == 'NarrativeText':
        s.add(len(do.page_content))
        print(do.page_content if len(do.page_content)<150 else "", end="\n")

Article

Academic Editor: Ohad Medalia


Received: 28 July 2022 Accepted: 23 August 2022 Published: 25 August 2022
Publisher’s Note: MDPI stays neutral with regard to jurisdictional claims in published maps and institutional affil- iations.


Cells 2022, 11, 2650. https: / /doi.org /10.3390/cells11172650
https: //www.mdpi.com/journal /cells


2.1. Electrical Stimulation (ES) Chamber C-Pace EM 100






2.2. Calculation and Simulation of ES


2.3. Current Measurement

2.4. Characterization of AC-Stimulated Liquid


For all Controls, the electrodes were placed in the medium, and the power switch was not returned on (no ES).
2.5. Cell Culture

2.6. AC-Activated Medium and Online Monitoring of Cell Adhesion


2.7. AC stimulation of Cells and Initial Adhesion

2.8. Cell Morphology and Spreading



2.9. Calcium Mobilization and AC stimulation


2.10. Cellular Reactive Oxygen Species (ROS)



2.11. Statistics


3.1. Characterization of AC Electrical Stimulation (ES) Parameter


8 of 22
Table 

In [6]:
# Remove boilerplate narrative texts
import re
from langchain_core.documents import Document
BOILERPLATE_PATTERN = r"""
\b\d+\s*of\s*\d+\b | # Matches '18 of 22'
\b[A-Za-z]+\s+\d{4},\s*\d+,\s*\d+\b # Matches 'Cells 2022, 11, 2650'
"""

boilerplate_regex = re.compile(BOILERPLATE_PATTERN, re.VERBOSE)
def remove_boilerplate(Docs: Document) -> list[Document]:
    for doc in Docs:
        if (doc.metadata.get('category') == 'NarrativeText'):
            if boilerplate_regex.fullmatch(doc.page_content.strip()):
                Docs.remove(doc)
    return Docs
            

In [7]:
# Remove unnnecessary narattivetext like citation publisher info

def clean_narrative_text(Docs: Document) -> list[Document]:
    text_start_with = ['Cells 2022, 11, 2650', 'Article', 'Academic Editor','Received:', 'Citation',
                       'Accepted:' , 'Published:',"Publisher’s Note:", "G check for updates", "Susanne Staehlke 1",
                       "Institutional Review Board Statement: Not applic",'Informed Consent Statement: Not applicable',
                       'Conﬂicts of Interest:', 'https:', 'Copyright: © ', 'Acknowledgments:', 'Funding:','Data Availability Statement','References']
    docs = [doc for doc in Docs if not any(doc.page_content.strip().startswith(text) for text in text_start_with)]
    return docs

In [8]:
Docs = remove_boilerplate(Docs)
Docs = clean_narrative_text(Docs)
len(Docs)
metadata_keys = set()
for doc in Docs:
    print(doc.page_content if doc.metadata.get('category') == 'Title' else "",)

Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling


1. Introduction



2. Materials and Methods







































3. Results






























3.4.4. Cell Morphology









4. Discussion



















5. Conclusions







In [21]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from dotenv import load_dotenv
import torch

# defining environment variable
load_dotenv()

# Defining Model
model_name = "meta-llama/Llama-3.1-8B-Instruct"

def open_souce_model(model_name, max_new_tokens: int):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    pipe = pipeline(
        "text-generation",
        model=model_name,
        tokenizer=tokenizer,
        device_map="auto",
        torch_dtype=torch.bfloat16, 
        max_new_tokens=max_new_tokens,   
        do_sample=False,          
        temperature=0.0,           
        repetition_penalty=1.2,
        return_full_text=False     
    )
    
    model_pipe = HuggingFacePipeline(pipeline=pipe)
    
    # Binding parameters to the runnable
    model = model_pipe.bind(skip_prompt=True) 
    
    return model

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser

def generate_table_summary(txt_as_html: str, caption: str) -> str:

    # defining a role for model
    Role = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert biomedical research assistant specializing in analyzing scientific tables. 
Your ONLY task is to generate a written summary of the provided table in exactly one paragraph of approximately 100 words.

**Strict Directives:**
1. **Focus on Comparisons:** Describe the relationship between the experimental groups (e.g., 1V vs 5V) and the control. 
2. **Handle Footnotes Safely:** Do not use math formulas (like E=U/d) to calculate new values. Mention that theoretical and measured values were compared if applicable, but focus on the resulting data.
3. **Data Precision:** For mean ± s.e.m. data (e.g., 319 ± 5.1), report only the mean (319) to maintain narrative flow.
4. **Professional Tone:** Use high-level scientific terminology (e.g., "osteoblastic proliferation," "amplitude-dependent response," "temporal increase").
5. **No Filler:** Start immediately with the summary. Do not use headers, bolding, or introductory phrases.

**Constraint:** If the summary is under 80 words, expand on the temporal trends (how data changed over time) or the impact of voltage/frequency variations.<|eot_id|><|start_header_id|>user<|end_header_id|>

Table Caption: [INSERT_CAPTION_HERE]
HTML Table: [INSERT_HTML_HERE]<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

    # setting up the parser. The summary will be in string only so, using string output parser
    #parser = PydanticOutputParser(pydantic_object=tablesummary)

    parser = StrOutputParser()
    # defining prompt
    table_template = ChatPromptTemplate([
        ("system", Role),
        ("human", "Please generate summary, Table content: \n{table_content}\nCaption: {table_caption}")
    ],
    )

    
    # defining a chain
    chain = table_template | model | parser
    response = chain.invoke({"table_content": txt_as_html, "table_caption": caption})
    return response

In [32]:
model = open_souce_model(model_name, max_new_tokens=300)

current_caption = None  # track latest caption

for doc in Docs:
    content = doc.page_content
    doc_category = doc.metadata.get("category", "")
    metadata_copy = doc.metadata.copy()

    # Step 1: Capture table caption
    if doc_category in {"NarrativeText", "FigureCaption"} and content.strip().startswith("Table"):
        current_caption = content.strip()
        continue

    # Step 2: Process table
    if doc_category == "Table" and metadata_copy.get("text_as_html"):
        table_html = metadata_copy["text_as_html"]

        summary = generate_table_summary(
            txt_as_html=table_html,
            caption=current_caption,
            
        )

        # Clean model artifacts
        summary = summary.replace("Assistant:", "").strip()

        print(summary)

        # Reset caption after use
        current_caption = None


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The asterisk indicates calculated parameters based on given settings. 

Summary:
The results show significant differences in electric current measurements across various setups when comparing the control group to those exposed to electrical stimulation at different frequencies and voltages. Notably, increasing the frequency from 7.9 Hz to 20 Hz resulted in higher measured currents for both 1V and 5V conditions, indicating an amplitude-dependent response. In contrast, varying the pulse duration had minimal effect on the measured current levels. These findings suggest that changes in either frequency or voltage significantly influence the magnitude of the induced electric field, leading to increased osteoblastic activity as observed through the measured electric current.
The results show that all stimulation conditions significantly increased cell area when compared to the unstimulated controls at each respective time point. The amplitude-dependent response was observed as higher voltage

### Key Notes: 
The above two functions will clean the complete NarativeText and Title category.

## Strategy 1: Parent-child chunking for Parent Document Retriever

In [19]:
# First check the what metadata keys are present
metadata_keys = set()
for doc in Docs:
    for key in doc.metadata.keys():
        metadata_keys.add(key)
print(metadata_keys)

{'text_as_html', 'parent_id', 'coordinates', 'detection_class_prob', 'page_number', 'languages', 'last_modified', 'file_directory', 'element_id', 'source', 'filename', 'filetype', 'category'}


### Generating parent chunks
> Generating parent chunks by diving document into sub-section wise chunks

In [16]:
## Creating sub-section wise chunks from cleaned documents
from uuid import uuid4
import re

def Create_parent_chunks(Docs: list[Document]) -> list[Document]:
    new_docs = []
    new_content = ""
    section_header = ""
    subsection_header = ""
    Paper_title = "Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling"
    metadata = {}
    current_chunk_page = None
    section_pattern = r"^(\d+\.)+\s*(.+)$"
    subsection_pattern = r"`\s*\d+(?:\.\d+)*\s+[^\n]+`"
    for doc in Docs:
        metadata_copy = doc.metadata.copy()
        
        content = doc.page_content
        current_page = metadata_copy.get('page_number', None)
        doc_category = doc.metadata.get('category', '')
        is_section_header = (doc_category == 'Title')

        if is_section_header:
            if new_content:
                chunk_metadata = doc.metadata.copy()
                chunk_metadata['section_header'] = section_header
                
                if current_chunk_page is not None:
                    chunk_metadata['start_page'] = current_chunk_page
                chunk_metadata['end_page'] = current_page
                chunk_metadata['paper_title'] = Paper_title
                document_id = str(uuid4())
                chunk_metadata['document_id'] = document_id
                new_docs.append(Document(page_content=new_content.strip(), metadata=chunk_metadata))

            if content.strip() != Paper_title:
                section_header = content.strip()
                new_content = ""
                metadata = metadata_copy
                current_chunk_page = current_page
            continue

        # Now we will add the category wise content, unless it is a section header
        if doc_category in ['NarrativeText', 'FigureCaption']:
            current_page = doc.metadata.get('page_number', None)
            chunk_metadata = doc.metadata.copy()
            chunk_metadata['section_header'] = section_header
            chunk_metadata['subsection_header'] = subsection_header
            if re.match(section_pattern, content.strip()):
                subsection_header = content.strip()
            new_content += "\n" + content.strip()+ "\n"

        if (doc_category == "NarrativeText" and content.startswith("Table")) or (doc_category == "FigureCaption" and content.startswith("Table")):
                caption = content.strip()

        if doc_category == "Table" and metadata_copy.get('text_as_html'):
                txt_html = metadata_copy.get('text_as_html')
                summary = generate_table_summary(txt_html, caption)
                new_content += "\n Table Summary: \n" + summary.replace("Assistant: ", "") + "\n"
                
        metadata.update(chunk_metadata)

    # After processing all documents, check if there's remaining content to be added
    if new_content:
        chunk_metadata = metadata.copy()
        chunk_metadata['section_header'] = section_header
        if current_chunk_page is not None:
            chunk_metadata['start_page'] = current_chunk_page
        chunk_metadata['end_page'] = current_page
        chunk_metadata['paper_title'] = Paper_title
        document_id = str(uuid4())
        chunk_metadata['document_id'] = document_id
        new_docs.append(Document(page_content=new_content.strip(), metadata=chunk_metadata))
    return new_docs



In [18]:
for i, doc in enumerate(new_docs):
    print(i, len(doc.page_content))

0 1568
1 3625
2 17195
3 12349
4 5193
5 15386
6 1989


In [20]:
# Load new_docs

import pickle

with open("pkl_files/parent_docs.pkl", "rb") as f:
    new_docs=pickle.load(f)

for doc in new_docs:
    print(doc.page_content)
    print("\n"+"---"*20+"\n" + str(doc.metadata)+"\n" +"---"*20 +"\n")

Abstract: An extensive research ﬁeld in regenerative medicine is electrical stimulation (ES) and its impact on tissue and cells. The mechanism of action of ES, particularly the role of electrical parameters like intensity, frequency, and duration of the electric ﬁeld, is not yet fully understood. Human MG-63 osteoblasts were electrically stimulated for 10 min with a commercially available multi-channel system (IonOptix). We generated alternating current (AC) electrical ﬁelds with a voltage of 1 or 5 V and frequencies of 7.9 or 20 Hz, respectively. To exclude liquid-mediated effects, we characterized the AC-stimulated culture medium. AC stimulation did not change the medium’s pH, temperature, and oxygen content. The H2O2 level was comparable with the unstimulated samples except at 5 V_7.9 Hz, where a signiﬁcant increase in H2O2 was found within the ﬁrst 30 min. Pulsed electrical stimulation was beneﬁcial for the process of attachment and initial adhesion of suspended osteoblasts. At the

In [21]:
for doc in new_docs:
    print(doc.metadata.get('document_id'))


0dd19c2d-1b2c-4cca-822a-df636d4b0b78
44aba20e-bd51-4964-950a-69aaf97680e9
7c7131c0-da93-4fc1-94e8-cd50f4417922
e986d6bb-50eb-40e6-b096-f643bd1b4f6c
56e66a30-b5c3-4bd5-b0dc-809be6b86b87
bcdc5b75-85d5-4543-9bab-7813f52b022d
26acc14b-3182-4be2-9818-4bc9ba8d0438


### Crating parent and child chunks

In [1]:
from transformers import AutoTokenizer
model_name="nvidia/llama-embed-nemotron-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def token_length(text: str) -> int:
    return len(tokenizer.encode(text, add_special_tokens=False))

/home/jovyan/work/CellSense/X20251016-CellSense-VB/cell/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for i, doc in enumerate(new_docs):
    print(i, token_length(doc.page_content))

NameError: name 'new_docs' is not defined

In [2]:
pip install --upgrade pydantic chromadb langchain-chroma langchain


  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.9
    Uninstalling pydantic-1.10.9:
      Successfully uninstalled pydantic-1.10.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.1 requires langchain-text-splitters<2.0.0,>=1.1.0, but you have langchain-text-splitters 0.2.4 which is incompatible.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 1.2.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [132]:
!pip freeze > requirements.txt

In [2]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.store.memory import InMemoryStore
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_core.documents import Document

parent_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, 
                                                 chunk_overlap =100,
                                                 separators=["\n\n", "\n", " ", ""],
                                                 length_function = token_length,
                                                 add_start_index = True)

child_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, 
                                                chunk_overlap = 20,
                                                length_function=token_length,
                                                add_start_index = True)





In [25]:
parent_chunks = parent_splitter.split_documents(new_docs)
child_chunks = child_splitter.split_documents(new_docs)
for p in parent_chunks:
    print(token_length(p.page_content), p.metadata.get('document_id'))


347 0dd19c2d-1b2c-4cca-822a-df636d4b0b78
762 44aba20e-bd51-4964-950a-69aaf97680e9
1952 7c7131c0-da93-4fc1-94e8-cd50f4417922
1909 7c7131c0-da93-4fc1-94e8-cd50f4417922
593 7c7131c0-da93-4fc1-94e8-cd50f4417922
1956 e986d6bb-50eb-40e6-b096-f643bd1b4f6c
1483 e986d6bb-50eb-40e6-b096-f643bd1b4f6c
1501 56e66a30-b5c3-4bd5-b0dc-809be6b86b87
1897 bcdc5b75-85d5-4543-9bab-7813f52b022d
1751 bcdc5b75-85d5-4543-9bab-7813f52b022d
609 26acc14b-3182-4be2-9818-4bc9ba8d0438


In [26]:
for p in child_chunks:
    print(token_length(p.page_content))
print("total child chunks= ",len(child_chunks))

347
271
266
224
384
380
384
304
394
372
278
314
94
399
250
395
356
242
375
228
360
227
388
394
191
351
366
384
237
215
307
171
307
225
293
248
328
300
388
78
336
217
221
285
373
399
64
391
37
185
398
43
total child chunks=  52


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name="nvidia/llama-embed-nemotron-8b"

embeddings = HuggingFaceEmbeddings(model_name=model_name,
                                      model_kwargs={'device': 0, "trust_remote_code": True} )



A new version of the following files was downloaded from https://huggingface.co/nvidia/llama-embed-nemotron-8b:
- llama_bidirectional_model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.37it/s]


In [4]:
vector_store = Chroma(embedding_function=embeddings,
                      collection_name="biorag_indexed_docs",  # Giving vectore store a name
                      persist_directory="./chroma_db/",  # Creeating a dirtectory to store the vecotr store in the computer
                      )

In [5]:
# Defining the orchestrator to manage parent-child relationship between documents
from langchain_community.docstore import InMemoryDocstore
from langchain_core.stores import InMemoryByteStore
# Two different stores for two jobs
byte_store = InMemoryByteStore()    # Embeddings → Fast search
docstore = InMemoryDocstore()       # Full docs → Rich context

retriever = ParentDocumentRetriever(
    vectorstore=vector_store,    # Uses byte_store internally
    docstore=byte_store,           # Full documents
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={"k": 3}
)



In [6]:
type(retriever)

langchain_classic.retrievers.parent_document_retriever.ParentDocumentRetriever

In [30]:
retriever.add_documents(new_docs)

In [31]:
# Retrieve Documents relevent to query

query = "Compare the simulated electric field outcomes with the measured electric current."
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

2
3. Results
4. Discussion


In [32]:
# Retrieve Documents relevent to query

query = "What is Cell Morphology and its significance in the study?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

3
3.4.4. Cell Morphology
4. Discussion
2. Materials and Methods


In [33]:
# Retrieve Documents relevent to query
# Target: Abstract/ Conclusion

query = "What is the key findings of the paper?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

2
5. Conclusions
4. Discussion


In [34]:
# Target: Abstract
query = "What was the main objective of this study regarding pulsed electrical stimulation and osteoblast behavior?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

3
4. Discussion
4. Discussion
1. Introduction


In [35]:
# Target: Introduction
query = "Why is electrical stimulation considered relevant for bone regeneration and osteoblast activity?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

3
4. Discussion
1. Introduction



In [36]:
# Target: Section 2.1 – Electrical Stimulation Chamber section 2
query = "In the Materials and Methods section, how is the IonOptix electrical stimulation chamber described, including electrode material, geometry, and well configuration?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

2
2. Materials and Methods
4. Discussion


In [37]:
# Target : Section 2.2 + 3.1
query = "How was the electric field strength between the electrodes calculated and validated in this study?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

2
3. Results
2. Materials and Methods


In [38]:
## The retrievd chunk should be from section 2 2.4 current measurements

query = "How was the electrical current measured during stimulation, and why was this measurement important?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

3
4. Discussion
3. Results
5. Conclusions


In [39]:
# Target: Section 2.4 + 3.2
query = "Which physicochemical properties of the culture medium were analyzed after AC stimulation, and what changes were observed?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

3
3. Results
5. Conclusions
4. Discussion


In [40]:
# Target: Section 2.5 – Cell Culture

query = "Why were MG-63 osteoblast-like cells chosen, and how were they prepared for the experiments?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

2
2. Materials and Methods
2. Materials and Methods


In [41]:
#Target: Sections 3.4.2 & 3.4.3- Calcium signaling results

query = "How did pulsed electrical stimulation affect the initial adhesion of suspended osteoblasts?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

3
2. Materials and Methods
3. Results
4. Discussion


In [42]:
#target: Section 3.4.5 – ROS Production

query = "What effect did AC electrical stimulation have on intracellular calcium levels and ATP-induced calcium mobilization?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

2
3. Results
4. Discussion


In [43]:
#Target:  Discussion section

query = "How do the authors explain the relationship between electrical stimulation, calcium signaling, and osteoblast activation?"
relevent_docs = retriever.invoke(query)
print(len(relevent_docs))
for i in relevent_docs:
    print(i.metadata.get('section_header'))

3
1. Introduction
4. Discussion



#### In above evaluation, i was cheking if the retriever gets the most relevent chunks from the document or not. Here is what I observed:
* The discussion section contains information for materials section as well as result section.
* Even when I ask question related to 2. section it retrieves discussion section.
* Most favorable approach here is to use advanced RAG techniques called query re-writting.
* Below in few experiments I will try to do few shot and zero shot query re-writting(prompting technique)

## Strategy- 2 : Rewrite-Retrieve-Read method

### Zero shot- Rewriting

In [44]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from dotenv import load_dotenv

# defining environment variable
load_dotenv()

model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation",
                tokenizer=tokenizer,
               model= model_name,
                device_map = "auto",
                dtype = "auto",
                max_new_tokens=300,   
                do_sample=False,
                temperature = 0.4,
                repetition_penalty=1.2
               )

model_pipe = HuggingFacePipeline(pipeline=pipe)
model = model_pipe.bind(skip_prompt = True,
                       stop=["\n\n\n", "Note:", "Query 4:", "Input:"])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [45]:
query_rewriting_system_prompt = """

You are a document-aligned technical query generator for the Bioelectromagnetics and Cell Biology domain.

Task:
Your task is to rewrite the user query for better search retrieval. Do not change the core subject or intent.

Primary objective:
The generated queries must help retrieve the SAME document sections as the original query.
Do NOT expand, generalize, or biologically enrich the query beyond the document’s scope.

Alignment constraints (MANDATORY):
- Preserve the original intent and level of abstraction of the input query.
- Do NOT introduce new cell types, organisms, tissues, or biological models not explicitly present or clearly implied in the input.
- Do NOT replace electrical stimulation with electromagnetic, magnetic, or clinical modalities unless explicitly stated.
- Do NOT shift from background or mechanistic questions to outcome-based, clinical, or experimental-result questions unless explicitly requested.
- Do NOT introduce new devices, procedures, or application contexts.

Semantic constraints:
- Queries must remain conservative and document-grounded.
- Prefer general mechanistic or explanatory phrasing over specific experimental outcomes.
- Do not speculate beyond concepts present in the input.

Terminology constraints:
- Do NOT rephrase, expand, or reinterpret technical acronyms.
- Use only terminology that is present or directly implied by the input query.

Output constraints:
- Output ONLY the queries.
- Exactly 3 queries.
- One query per line.
- No numbering, no bullets.
- No explanations, notes, headers, or additional text.

CRITICAL: You must return ONLY a JSON object. 
Do not include any text before or after the JSON.
Example format:
{{"queries": ["query 1", "query 2", "query 3"]}}

Failure condition:
If any text other than exactly three standalone queries is produced, the output is invalid.



Input Query: {input_query}



STRICT RULE:
Return only the three queries, each on its own line.



"""

In [46]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class query_parser(BaseModel):
    queries: List[str] = Field(description="A list of exactly 3 rewritten technical queries.")

QueryParser = PydanticOutputParser(pydantic_object = query_parser)
query_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", query_rewriting_system_prompt),
    ]
).partial(format_instructions=QueryParser.get_format_instructions())


query_chain = query_prompt | model | QueryParser

In [104]:

reponse = query_chain.invoke({
    "input_query": "How was the electric field strength between the electrodes calculated and validated in this study?"
})

print(reponse)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


queries=['How was the electric field strength between the electrodes determined?', 'The calculation method used for electric field strength between electrodes', 'What validation methods were employed for electrode spacing?']


In [48]:

reponse = query_chain.invoke({
    "input_query": "Why is electrical stimulation considered relevant for bone regeneration and osteoblast activity?"
})

print(reponse)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


queries=['Why does electrical stimulation influence bone regeneration?', 'What role does electrical stimulation play in promoting osteoblast activity?', 'How can electrical stimulation impact osteogenesis?']


In [49]:
query = "Which physicochemical properties of the culture medium were analyzed after AC stimulation, and what changes were observed?"
reponse = query_chain.invoke({
    "input_query":query
})
reponse.queries

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['What physicochemical properties of the culture medium were measured post-AC stimulation?',
 'The effects of AC stimulation on which culture medium properties have been studied?',
 "What alterations occurred in the culture medium's physical/chemical characteristics following AC exposure?"]

### Query rewritting is successfully done. 

## Step-2 Retrieve

* Parallel Search: Run each query against your vector database independently.

* Aggregation: Collect all the document chunks returned by the three searches.

* Deduplication: Remove duplicate chunks (since different queries might find the same paragraph).

In [105]:
## Parallel serach and Aggregation

re_chunks= []
for query in reponse.queries:
    retrieved_docs = retriever.invoke(query)
    re_chunks.extend(retrieved_docs)

## Deduplication
d=dict()
for cnks in re_chunks:
    docs_id = cnks.metadata.get('document_id')
    d[docs_id] = cnks
    
print(d.keys())
for k in d.keys():
    doc = d.get(k).metadata.get('section_header')
    print(doc)


dict_keys(['e986d6bb-50eb-40e6-b096-f643bd1b4f6c', 'bcdc5b75-85d5-4543-9bab-7813f52b022d', '7c7131c0-da93-4fc1-94e8-cd50f4417922'])
3. Results
4. Discussion
2. Materials and Methods


* Here, All those queries have retrieved 5 parent documents for one user query
* Each parent document has almost 2000 token length.
* Model's contrxt window is limited and all these documnets combines with approximately 10000 tokens.
* The issue, stuffing 10,000 tokens of dense technical data into a single prompt, can lead to "Lost in the Middle" syndrome.
* The model ignores the middle documents and only focuses on the beginning and end.

## Strategy-3: Map-Reduce

### Mapping
* First, we take the all document and generate it's summaries too 400 tokens so all document's summary will be 2000 tokens.
* All the summary will relate to user query.

In [93]:
from langchain_core.prompts import ChatPromptTemplate

map_sys_msg = """
You are a Scientific Data Extraction Engine.

TASK:
Analyze the provided document text and determine whether it contains information relevant to the user query.
If relevant, extract and summarize the experimental findings in a single, detailed, informative, well-formed paragraph.

STRICT OUTPUT RULES (MANDATORY):
1. Output ONLY a single valid JSON object.
2. Output MUST start with '{{' and end with '}}'.
3. The JSON object must contain exactly two fields: "relevant" and "summary".
4. The "summary" field MUST be a SINGLE STRING containing ONE coherent paragraph.
5. The summary MUST describe all findings, observations, or understanding, and MUST NOT contain objects, dictionaries, lists, or key–value formatting inside it.
6. The summary should be up to 400–500 words.
7. Do NOT include explanations, notes, markdown, code, or any extra text outside the JSON.
8. If no relevant information is found, return EXACTLY:
   {{"relevant": false, "summary": "No relevant data found."}}

SCHEMA:
{{
  "relevant": boolean,
  "summary": string
}}
"""

In [52]:
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from dotenv import load_dotenv

# defining environment variable
load_dotenv()

model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation",
                tokenizer=tokenizer,
                   model= model_name,
                device_map = "auto",
                dtype = "auto",
                max_new_tokens=512,   
                do_sample=False,
                temperature = 0.4,
                repetition_penalty=1.2,
                return_full_text= True
               )

model_pipe = HuggingFacePipeline(pipeline=pipe)
map_model = model_pipe.bind(skip_prompt = True,)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [94]:
from langchain_core.runnables import RunnableLambda
import json

def normalize_llm_output(raw: str) -> str:
    """
    Extracts the FIRST valid JSON object from LLM output
    and returns it as a JSON STRING (LangChain-safe).
    """

    if not raw or not isinstance(raw, str):
        raise ValueError("LLM output is empty or not a string")

    text = raw.strip()
    text = text.replace("```json", "").replace("```", "")

    start = None
    brace_count = 0

    for i, ch in enumerate(text):
        if ch == "{":
            if start is None:
                start = i
            brace_count += 1
        elif ch == "}":
            brace_count -= 1
            if brace_count == 0 and start is not None:
                json_text = text[start:i+1]
                break
    else:
        raise ValueError("No JSON object found in LLM output")

    try:
        data = json.loads(json_text)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON parsing failed: {e}\nExtracted:\n{json_text}")

    # Schema hardening
    data.setdefault("relevant", False)
    data.setdefault("summary", "")

    if isinstance(data["summary"], dict):
        data["summary"] = " ".join(f"{k}: {v}" for k, v in data["summary"].items())

    data["relevant"] = bool(data["relevant"])
    data["summary"] = str(data["summary"])


    return json.dumps(data, ensure_ascii=False)

    return data


normalizer = RunnableLambda(normalize_llm_output)

In [95]:
from pydantic import BaseModel, Field

class Summariser(BaseModel):
    relevant: bool = Field(description="Whether the text contains info about the physicochemical properties or biological effects of the stimulation.")
    summary: str = Field(
        description="A single-paragraph summary of 500-700 words containing only the information from the text that directly addresses the user query."
    )


# This parser creates the format instructions that we inject into the prompt
map_parser = PydanticOutputParser(pydantic_object=Summariser)


map_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", map_sys_msg),
        ("user", "Query: {user_query}\nText: {text_chunk}")
    ]
).partial(format_instructions=map_parser.get_format_instructions())
map_chain = map_prompt | map_model | normalizer | map_parser

In [106]:
for x in list(d.keys())[:1]:
    do = d.get(x)
do

Document(metadata={'source': 'Documents/cells-11-02650-v2.pdf', 'detection_class_prob': 0.7357485890388489, 'is_extracted': 'true', 'last_modified': '2025-12-02T10:31:49', 'filetype': 'application/pdf', 'page_number': 13, 'file_directory': 'Documents', 'filename': 'cells-11-02650-v2.pdf', 'category': 'Title', 'element_id': '55d05466ab31cd967316cbb1702b4345', 'section_header': '3. Results', 'start_page': 8, 'end_page': 13, 'paper_title': 'Pulsed Electrical Stimulation Affects Osteoblast Adhesion and Calcium Ion Signaling', 'document_id': 'e986d6bb-50eb-40e6-b096-f643bd1b4f6c', 'start_index': 0}, page_content='3.1. Characterization of AC Electrical Stimulation (ES) Parameter\n\nThe electric ﬁeld strength and its magnitude between the electrodes can be seen in Figure 3. Between the electrodes, where the cells are placed, is a large region with a more or less homogeneous ﬁeld. Only close to the electrode edges a ﬁeld enhancement can be observed (Figure 3a). However, the area with an enhanc

In [107]:
query = "How was the electric field strength between the electrodes calculated and validated in this study"
map_response = map_chain.invoke({"user_query":query,
                        "text_chunk": do})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [108]:
print(map_response.summary)

To calculate the electric field strength between the electrodes, simulations were performed assuming infinitely wide capacitor plates, which yielded a field strength of 90.91 V/m under a voltage difference of 1 V and a spacing of 1.1 cm. This analytical estimation matched closely with the numerically modeled field strength of around 90 V/m along the centerline between the electrodes. Validation of the numerical model involved comparing the simulated electric field with the measured current through the electrode, which equaled the conductivity times the normal component of the electric field integrated over the electrode surface. The results indicated that the cells experienced a mostly homogeneous electric field with a magnitude of about 90 V/m. Additionally, measurements confirmed that increasing the voltage and frequency led to higher currents, although the measured values were consistently lower than the theoretically predicted ones. Further analysis focused on characterizing the ac

In [109]:
token_length(map_response.summary)

256

### Writing functions to automate the mapping

In [114]:
# function for query rewriting
query = "How was the electric field strength between the electrodes calculated and validated in this study?"
def rewrite_query(user_query:str) -> List[str]:

    # Defining chain
    query_chain = query_prompt | model | QueryParser

    # invoking the chain
    queries = query_chain.invoke({"input_query": user_query})
    return queries
queries = rewrite_query(query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [115]:
# Function for collecting rlevent document based on re written queries
from langchain_core.documents import Document
def relevent_retrieved_docs(list_of_queries: List[str]) -> List[Document]:
    retrieved_chunks= []
    relevent_documents=dict()
    for query in list_of_queries.queries:
        relevent_doc = retriever.invoke(query)
        retrieved_chunks.extend(relevent_doc)

    for chunk in retrieved_chunks:
        doc_id = chunk.metadata.get('document_id')
        relevent_documents[doc_id] = chunk

    return relevent_documents
rel_doc = relevent_retrieved_docs(queries)   

In [116]:
type(rel_doc)

dict

In [117]:
def combined_summary(user_query: str, list_of_relevent_docs: dict):
    
    map_chain = map_prompt | map_model | normalizer | map_parser
    combined_summaries = []
    for doc in list_of_relevent_docs:
        doc_obj = list_of_relevent_docs[doc]
        response = map_chain.invoke({
            "user_query": user_query, 
            "text_chunk": doc_obj
        })
        
        # Accessing the content safely (handling different model output types)
        summary_text = response.summary if hasattr(response, 'content') else str(response.summary)
        
        # Simple filter to prevent hallucination carry-over
        if response.relevant:
            combined_summaries.append(summary_text)

    # Join with clear delimiters so the final reduce step knows where chunks end
    return "\n\n".join(combined_summaries)

summary = combined_summary(user_query=query, list_of_relevent_docs=rel_doc)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [118]:
import pickle
with open("pkl_files/summary.pkl",'wb') as f:
    pickle.dump(summary, f)

In [119]:
import pickle
with open("pkl_files/summary.pkl",'rb') as f:
    summary = pickle.load(f)
summary

"To calculate the electric field strength between the electrodes, simulations were performed assuming infinitely wide capacitor plates, which yielded a field strength of 90.91 V/m under a voltage difference of 1 V and a spacing of 1.1 cm. This analytical estimation matched closely with the numerically modeled field strength of around 90 V/m along the centerline between the electrodes. Validation of the numerical model involved comparing the simulated electric field with the current flowing through the electrodes, which equaled the conductivity times the normal component of the electric field integrated over the electrode surface, giving 1.75 A/m for the 2D geometry. Experimental measurements confirmed that the electric field decreased from 91.7 V/m at the electrodes to 90.2 V/m at the midpoint between them. Higher voltages led to stronger electric fields, while increasing frequencies caused larger currents. Additionally, the analysis of acellular medium characteristics revealed signifi

In [120]:
reduce_system_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a Senior Research Data Auditor. Your task is to synthesize provided summaries into a structured technical narrative while maintaining absolute factual integrity.

### NARRATIVE REQUIREMENTS (The "analysis" field):
1. **Factual Anchor**: Use only the information explicitly stated in the source text. 
2. **Qualitative vs. Quantitative**: If the source provides specific numbers or units, include and **bold** them. If the source uses descriptive language (e.g., "enhanced," "decreased," "significant"), you MUST mirror that language. Never convert qualitative descriptions into numerical estimates.
3. **Causal Logic**: Explain the relationship between the inputs/conditions and the resulting outcomes based solely on the provided evidence.
4. **Seamless Style**: Write in professional paragraphs without headers. Length should be proportional to the actual data provided.

### DATA EXTRACTION RULES (The "key_metrics" field):
- **STRICT FACTUALITY**: Only extract values that appear verbatim in the source.
- **NO INFERENCE**: Do not calculate, estimate, or "fill in" missing values (e.g., do not invent a +/- range if none is given).
- **Empty State**: If no explicit numerical measurements or specific named parameters are found, return "key_metrics": [].

### FORMATTING:
- Output MUST be a valid JSON object.
- The "analysis" field must be a single string. Use \\n for paragraph breaks.

<|eot_id|><|start_header_id|>user<|end_header_id|>
Query: {user_query}
Source Data:
{map_summaries}

### RESPONSE (JSON ONLY):
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [65]:
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from dotenv import load_dotenv

# defining environment variable
load_dotenv()

model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation",
                tokenizer=tokenizer,
                model= model_name,
                device_map = "auto",
                dtype = "auto",
                max_new_tokens=1024,   
                do_sample=False,
                temperature = 0.5,
                repetition_penalty=1.2,
                
               )

model_pipe = HuggingFacePipeline(pipeline=pipe)
reduce_model = model_pipe.bind(skip_prompt = True,)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [121]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional

class Metric(BaseModel):
    parameter: Optional[str] = Field(
        default=None,
        description="What was measured, if explicitly stated"
    )
    value: Optional[str] = Field(
        default=None,
        description="Measured value with unit, if available"
    )

class ResearchSynthesisParser(BaseModel):
    is_sufficient: Optional[bool] = Field(
        default=None,
        description="True if data was found."
    )

    key_metrics: Optional[List[Metric]] = Field(
        default=None,
        description="Optional list of extracted measurements."
    )

    analysis: str = Field(
        description="The seamless narrative report (no headers) as multiple paragraphs."
    )

ReduceParser = PydanticOutputParser(pydantic_object = ResearchSynthesisParser)

In [122]:
import json
from langchain_core.runnables import RunnableLambda

def normalize_research_output(raw_output) -> str:
    """
    Normalizes raw LLM outputs for CellSense:
    - Removes problematic backslashes (\)
    - Ensures valid JSON
    - Enforces schema: analysis (str), key_metrics (list[dict])
    """

    # 1️⃣ If already a dict, just use it
    if isinstance(raw_output, dict):
        data = raw_output

    # 2️⃣ If it's a string, clean it
    elif isinstance(raw_output, str):
        raw_output = raw_output.strip()

        # Remove leading/trailing quotes
        if (raw_output.startswith("'") and raw_output.endswith("'")) or \
           (raw_output.startswith('"') and raw_output.endswith('"')):
            raw_output = raw_output[1:-1]

        #  Remove all backslashes (this fixes \_ and any others)
        raw_output = raw_output.replace("\\", "")

        try:
            data = json.loads(raw_output)
        except json.JSONDecodeError:
            return json.dumps({
                "is_sufficient": False,
                "key_metrics": [],
                "analysis": "JSON parsing failed"
            })

    else:
        return json.dumps({
            "is_sufficient": False,
            "key_metrics": [],
            "analysis": "Unsupported output type"
        })

    # 3️⃣ Schema hardening
    if not isinstance(data.get("analysis"), str):
        data["analysis"] = str(data.get("analysis", ""))

    if not isinstance(data.get("key_metrics"), list):
        data["key_metrics"] = []

    # Ensure each metric is a dict
    data["key_metrics"] = [m for m in data["key_metrics"] if isinstance(m, dict)]

    return json.dumps(data, ensure_ascii=False)


research_normalizer = RunnableLambda(normalize_research_output)


In [123]:
from langchain_core.prompts import ChatPromptTemplate

reduce_template = ChatPromptTemplate(
    [
        ("system",  reduce_system_prompt)
    ]
).partial(format_instructions=ReduceParser.get_format_instructions())

In [129]:
reduce_chain = reduce_template | reduce_model 

In [130]:
#query = "How did pulsed electrical stimulation affect the initial adhesion of suspended osteoblasts?"
query = "How was the electric field strength between the electrodes calculated and validated in this study?"
Final_response = reduce_chain.invoke({
    "user_query": query,
    "map_summaries": summary
})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [131]:
Final_response

'{\n    "analysis": "\\nThe electric field strength between the electrodes was first estimated analytically, yielding a value of **90.91 V/m**, considering infinite plate capacitors under a **1 V** voltage difference and a distance of **1.1 cm**.\\nThis analytical approximation aligned closely with the computed electric field strength via numerical modeling, approximately **90 V/m**, measured along the central axis connecting the electrodes.\\nValidation of the simulation entailed matching it against the current passing through the electrodes, amounting to **1.75 A/m** for a two-dimensional setup.\\nExperimental observations indicated that the electric field diminished gradually from **91.7 V/m** at the electrodes down to **90.2 V/m** midway between them.\\nHigher applied voltages resulted in more robust electric fields, while elevated frequencies produced greater currents.\\nMoreover, detailed examination of the properties of the test environment disclosed notable alterations in tempe

In [1]:
!nvidia-smi

Failed to initialize NVML: Unknown Error


### Generation

In [67]:
system_msg = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an Expert Scientific Literature Analyst. Your goal is to provide concise, factual, evidence-based answers.

--- INSTRUCTIONS ---
1. STRICT: Answer the user's question ONLY using the text in the CONTEXT section below.
2. EVIDENCE: If the answer is found, include the source document by using its 'source' metadata (e.g., [Source: document_name.pdf, page 5]).
3. UNCERTAINTY: If the answer cannot be found in CONTEXT, respond: 'I apologize, the necessary information could not be found in the available document sections.'
4. FORMAT: Use clear, structured output (bullet points, numbered lists, or short paragraphs).

--- CONTEXT ---
{context}

<|eot_id|><|start_header_id|>user<|end_header_id|>

"""

In [68]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


parser = StrOutputParser()
template =  ChatPromptTemplate.from_messages(
    [
        (
            "system", system_msg
        ),
        ("user", "{question}"),
    ]
)



In [69]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from dotenv import load_dotenv

# defining environment variable
load_dotenv()

model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation",
                tokenizer=tokenizer,
               model= model_name,
                device_map = "auto",
                dtype = "auto",
                max_new_tokens=700,   
                do_sample=False,
                temperature = 0.2,
                repetition_penalty=1.5
               )

model_pipe = HuggingFacePipeline(pipeline=pipe)
model = model_pipe.bind(skip_prompt = True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]
Device set to use cuda:0


In [70]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

parallel_chain = (
    {
    "context": retriever,
    "question": RunnablePassthrough()
    }   
)

chain = parallel_chain | template | model | parser

In [72]:
response = chain.invoke("How was the electrical current measured during stimulation, and why was this measurement important?")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 

Answer:
According to Section "Characteristics", subsection “Measurement” isn’t present however looking into another relevant context - Subection ‘‘Electric field’’ measurements weren't mentioned explicitly related to how they measure electricity flow. But rather what’s interesting lies elsewhere :  



Section ''Results'' > sub-section ``Accllar Media Charactristics — Ac Stimulatoin ''. Here you find out details regarding measuring eletrical Current.



Here is your detailed explanation:

To determine the validity of simulations used throughout research paper,
they needed accurate measures of Electric Field Strength
and compare them against those predicted theoretically.


They utilized `Tekronik DTS210` oscilloscope device specifically designed for high-speed digital storage scopes capable capturing fast transient signals accurately recording waveforms quickly making precise timing easy-to-read displays allowing users access real-world insights instantly saving valuable lab space s

In [ ]:
response = chain.invoke("Why did the authors specifically select the frequencies 7.9 Hz and 20 Hz, and how do these frequencies relate to physiological electrical signals in bone?")
print(response)

In [33]:
response = chain.invoke("How do the authors distinguish between direct electrical effects on cells and indirect effects mediated through changes in the culture medium? ")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 --- 

---

## Step 1
To determine how the authors differentiate between immediate impacts caused straight away onto biological entities versus secondary influences brought forth indirectly upon those organisms.
 
First off let’s look if there exists anywhere throughout given context where researchers discuss what exactly constitutes "direct" vs “indirect” influence.


Upon reviewing said passage one finds out - ‘Direct’ refers here specifically towards alterations happening right immediately without need intermediate substances whereas anything else would fall underneath category labeled simply put – Indirection.



Now since now know definitions lets proceed analyzing actual experiment setup details mentioned later down lines describing certain procedures carried-out involving application external stimuli followed closely afterwards measuring resultant outcomes produced thus enabling us infer differences existing possibly arising either path taken



Given description clearly outline

* The model behaving expectedly but the output generation shows too much latency. Also, the answers needed to be complete an non-repetative.
* I obserevded that, if the generated answer is shorter than max_tokens then it repeats the answer and if the answer is longer than max_token it generates incomplete answer.
* This observation shows we need to fine-tune the model to behave and reponse in specific way.

* No ReRanking is necessary as Here is only one document and has only few sections. If later adding more documents then Reranking might be necessary to choose from the extracted text.

### Finetunning -LoRA (GPT-OSS-20B)

#### Dataset (PubMedQA + BioInstruct)

* To generate answer in gpt-oss format I am finetunig the model using LoRA on PubMedQA dataset
* The dataset need to converted in the form of [openAI harmony](https://cookbook.openai.com/articles/openai-harmony#concepts), because openai oss model has been trained to generate response in that prompt format.

In [35]:
from datasets import load_dataset


bioins_ds = load_dataset("bio-nlp-umass/bioinstruct")
pubmed_ds = load_dataset("qiaojin/PubMedQA", "pqa_artificial")

In [36]:
pubmed_dataset = pubmed_ds['train'].shuffle(seed=42).select(range(2000))
pubmed_dataset.column_names

['pubid', 'question', 'context', 'long_answer', 'final_decision']

In [37]:
bioins_dataset = bioins_ds['train'].shuffle(seed=42).select(range(3000))
bioins_dataset.column_names

['instruction', 'input', 'output']

#### Creating a function that reformates dataset

* The given PubMedQA dataset has question context, and answer but it doesn't have instruction that can be used in analysis channel of harmony template.
* I am creating random analysis channel messages for model to use so, that it does not have identical thinking for each query.
* FORMAT:```
<|channel|>analysis<|message|>User asks: "What is 2 + 2?" Simple arithmetic. Provide answer.<|end|>
<|start|>assistant<|channel|>final<|message|>2 + 2 = 4.<|return|>```

In [38]:
import random
def reformat_to_harmony(dataset):

    # randomly selecting the reasoning phrases to prevent overfitting
    starts = ["Assessing the impact of {topic} on patient outcomes.",
              "Investigating mechanistic insights related to {topic}.",
              "Reviewing recent findings regarding {topic} interventions.",
              "Evaluating the reported correlations between biomarkers and {topic}.",
              "Summarizing evidence from the study concerning {topic}.",
              "Identifying methodological approaches used to study {topic}.",
              "Interpreting clinical significance of results related to {topic}.",
             "Analyzing clinical query regarding {topic}.",
            "Reviewing provided medical abstract for evidence on {topic}.",
            "Identifying key variables in the study of {topic}.",
            "Examining the relationship between the study population and {topic}."
    ]

    second_step = [
        "Cross-referencing provided context for statistical significance and p-values.",
        "Evaluating the experimental methodology and findings in the abstract.",
        "Synthesizing the results presented in the provided medical text.",
        "Checking for potential bias or study limitations in the context.",
        "Comparing reported findings with established literature on {topic}.",
        "Analyzing dosage, timing, or exposure variables in relation to {topic}.",
        "Verifying statistical robustness and confidence intervals in the reported data.",
        "Assessing whether conclusions are supported by the experimental evidence.",
        "Highlighting key trends or patterns observed in the study results.",
        "Checking for consistency between figures, tables, and text in the context.",
        "Identifying potential confounding factors affecting the interpretation of {topic}."
    ]

    
    # creating a topic from the question
    question_starters = ["Is", "Can", "Does", "Do", "Are"]
    topic = " ".join((dataset['question'].split()[:]))
    for st in question_starters:
        if topic.startswith(st):
            topic = topic.replace(st,"")
            topic = topic.replace("?","")

    # making analysis block message
    reasoning = (f"{random.choice(starts).format(topic=topic)} "
        f"{random.choice(second_step)} "
        f"Conclusion aligns with: {dataset['final_decision']}.")

    # Building the harmony format
    system_msg = (
        "<|start|>system<|message|>\n"
        "reasoning_effort: high\n"
        "# Valid channels: analysis, final.\n"
        "You are a Biomedical Research Assistant. Your role is to help researchers with necessary details asked in question."
        "by accurately synthesizing literature and data. Use 'analysis' to evaluate "
        "the evidence and 'final' for your structured response.\n"
        "<|end|>\n"
    )

    user_msg = f"<|start|>user<|message|>\nQUESTION: {dataset['question']}\nCONTEXT: {dataset['context']}\n<|end|>\n"

    analysis_block = f"<|start|>assistant<|channel|>analysis<|message|>\n{reasoning}\n<|end|>\n"

    final_block = f"<|start|>assistant<|channel|>final<|message|>\n{dataset['long_answer']}\n<|end|>"

    return {"text": system_msg + user_msg + analysis_block + final_block}


In [39]:
formated_pubmed_ds = pubmed_dataset.map(reformat_to_harmony)

In [40]:
formated_pubmed_ds[0]['text']

'<|start|>system<|message|>\nreasoning_effort: high\n# Valid channels: analysis, final.\nYou are a Biomedical Research Assistant. Your role is to help researchers with necessary details asked in question.by accurately synthesizing literature and data. Use \'analysis\' to evaluate the evidence and \'final\' for your structured response.\n<|end|>\n<|start|>user<|message|>\nQUESTION: Are keratin 8 Y54H and G62C mutations associated with inflammatory bowel disease?\nCONTEXT: {\'contexts\': [\'Keratin 8 is a major component of intermediate filaments in single-layered epithelia of the gastrointestinal tract. Keratin 8 deficient mice display signs of colitis and diarrhoea characteristic for inflammatory bowel disease. Very recently, two keratin 8 mutations, Y54H and G62C, were identified.\', \'We investigated if these keratin 8 missense mutations were associated with inflammatory bowel disease.\', "In total, 217 German patients with Crohn\' s disease, 131 German patients with ulcerative colit

#### Now with BioInstruct dataset

In [41]:
s= set()
for i in bioins_dataset['instruction']:
    s.add(i.split()[0])

s

{'4.',
 'Advise',
 'Analyze',
 'Answer',
 'Ask',
 'Assess',
 'Assist',
 'Based',
 'Break',
 'Briefly',
 'Calculate',
 'Can',
 'Categorize',
 'Choose',
 'Classify',
 'Compare',
 'Complete',
 'Considering',
 'Contrast',
 'Convert',
 'Correct',
 'Create',
 'Critically',
 'Decide',
 'Define',
 'Dentists',
 'Describe',
 'Design',
 'Detect',
 'Determine',
 'Develop',
 'Differentiate',
 'Discuss',
 'Draft',
 'Elaborate',
 'Estimate',
 'Evaluate',
 'Explain',
 'Explain,',
 'Express',
 'Extract',
 'Find',
 'For',
 'Formulate',
 'From',
 'Generate',
 'Give',
 'Given',
 'Help',
 'Highlight',
 'How',
 'Identify',
 'In',
 'Indicate',
 'Interpret',
 'Is',
 'List',
 'Looking',
 'Make',
 'Mention',
 'Name',
 'Offer',
 'Organize',
 'Outline',
 'Paraphrase',
 'Parse',
 'Perform',
 'Please',
 'Predict',
 'Prepare',
 'Present',
 'Propose',
 'Provide',
 'Provided',
 'Providing',
 'Rank',
 'Re-phrase',
 'Re-write',
 'Read',
 'Rearrange',
 'Recognize',
 'Recommend',
 'Request',
 'Research',
 'Restate',
 'Rev

In [42]:
def reformating_bioins(dataset):
    instruction = dataset['instruction']
    first_word = instruction.split()[0].rstrip(',.:')

    ## In this dataset i dont have question to format the dataset in harmony, so using instructions 
    ## and taking it's intent like generate, describe and convert it into summarizing, generating.
    
    # defining intent group
    intent_map = {
        # Commands & Actions
        "Summarize": "Summarizing", "Analyze": "Analyzing", "Explain": "Explaining",
        "Generate": "Generating", "Extract": "Extracting", "Compare": "Comparing",
        "Describe": "Describing", "Evaluate": "Evaluating", "Classify": "Classifying",
        "Calculate": "Calculating", "Predict": "Predicting", "Interpret": "Interpreting",
        
        # Requests & Polite starts
        "Please": "Processing request", "Can": "Assessing capability", 
        "Ask": "Responding", "Suggest": "Formulating suggestion",
        
        # Contextual & Conditional starts
        "Given": "Analyzing provided data", "Considering": "Weighing factors", 
        "Based": "Evaluating based on evidence", "With": "Reviewing context",
        
        # Question-based starts
        "What": "Identifying", "How": "Detailing process", "Which": "Selecting",
        "Is": "Verifying"
    }

    ## mapping, and if the first word is not present in map we replace it with following
    action = intent_map.get(first_word,"Addressing biomedical task")

    # building analysis message block
    reasoning = (
        f"{action} for the research query. "
        "Step 1: Identifying key medical entities and constraints. "
        "Step 2: Synthesizing response based on biomedical standards."
    )
    
    # Building the harmony format
    system_msg = (
        "<|start|>system<|message|>\n"
        "reasoning_effort: high\n"
        "# Valid channels: analysis, final.\n"
        "You are a Biomedical Research Assistant. Your role is to help researchers with necessary details asked in question."
        "by accurately synthesizing literature and data. Use 'analysis' to evaluate "
        "the evidence and 'final' for your structured response.\n"
        "<|end|>\n"
    )

    user_msg = f"<|start|>user<|message|>\nQUESTION: {dataset['instruction']}\nCONTEXT: {dataset['input']}\n<|end|>\n"
    analysis_block = f"<|start|>assistant<|channel|>analysis<|message|>\n{reasoning}\n<|end|>\n"
    final_block = f"<|start|>assistant<|channel|>final<|message|>\n{dataset['output']}\n<|end|>"
    
    return {"text": system_msg + user_msg + analysis_block + final_block}

In [43]:
formated_bioins_ds=bioins_dataset.map(reformating_bioins)

In [44]:
len(formated_pubmed_ds)

2000

In [45]:
# Merging dataset

def merge(pub, bio):
    lis = {pub[i]['text'] for i in range(len(pub))}
    for j in range(len(bio)):
        lis.add(bio[j]['text'])
    return lis

full_dataset = merge(pub=formated_pubmed_ds, bio=formated_bioins_ds)
len(full_dataset)
    
    

5000

In [46]:
from datasets import Dataset

ds = Dataset.from_dict({"text":full_dataset})

In [47]:
ds.push_to_hub("vrjb/BioRAG")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 33.42ba/s]
Processing Files (0 / 0): |          |  0.00B /  0.00B            
Processing Files (0 / 1):  15%|█▍        |  528kB / 3.55MB,  880kB/s  
Processing Files (0 / 1):  45%|████▍     | 1.58MB / 3.55MB, 1.59MB/s  
Processing Files (0 / 1):  89%|████████▉ | 3.17MB / 3.55MB, 2.64MB/s  
Processing Files (1 / 1): 100%|██████████| 3.55MB / 3.55MB, 2.22MB/s  
Processing Files (1 / 1): 100%|██████████| 3.55MB / 3.55MB, 1.97MB/s  
New Data Upload: 100%|██████████| 3.55MB / 3.55MB, 1.97MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.62s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Vrjb/BioRAG/commit/172c6f3237a51d4c83ca514f3c65d6e2bfe27ad5', commit_message='Upload dataset', commit_description='', oid='172c6f3237a51d4c83ca514f3c65d6e2bfe27ad5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Vrjb/BioRAG', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Vrjb/BioRAG'), pr_revision=None, pr_num=None)

### I have created and merged two dataset to fintune the GPT-OSS model in to required prompt template

# Fine-Tunning Using LoRA
* [finetune with gpt-oss and HF](https://cookbook.openai.com/articles/gpt-oss/fine-tune-transfomers)

In [48]:
## Loading the dataset

from datasets import load_dataset

dataset = load_dataset("vrjb/BioRAG", split='train')
dataset[0]

Generating train split: 100%|██████████| 5000/5000 [00:00<00:00, 95600.16 examples/s]


{'text': "<|start|>system<|message|>\nreasoning_effort: high\n# Valid channels: analysis, final.\nYou are a Biomedical Research Assistant. Your role is to help researchers with necessary details asked in question.by accurately synthesizing literature and data. Use 'analysis' to evaluate the evidence and 'final' for your structured response.\n<|end|>\n<|start|>user<|message|>\nQUESTION: Explain how an MRI machine works in simple terms.\nCONTEXT: MRI (Magnetic Resonance Imaging) is an imaging technique that uses magnetic fields and radio waves to create detailed images of the inside of the body. It works by aligning hydrogen atoms in the body with a strong magnetic field and then applying radiofrequency pulses. The atoms emit signals, which are detected by a scanner and processed to create an image.\n<|end|>\n<|start|>assistant<|channel|>analysis<|message|>\nExplaining for the research query. Step 1: Identifying key medical entities and constraints. Step 2: Synthesizing response based on

In [49]:
# Loading Tokenizer

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

# Applying chat template methhod to format the messages

messages = dataset[0]["text"]
conversation = tokenizer.apply_chat_template(messages, tokenize=False)
print(conversation)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2026-01-07

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|>


In [2]:
!nvidia-smi

Failed to initialize NVML: Unknown Error


In [51]:
# Preparing the model

import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="eager",
    dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

Loading checkpoint shards:  67%|██████▋   | 2/3 [00:08<00:04,  4.06s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.98 GiB. GPU 0 has a total capacity of 79.25 GiB of which 1.52 GiB is free. Including non-PyTorch memory, this process has 63.43 GiB memory in use. Of the allocated memory 60.30 GiB is allocated by PyTorch, and 2.63 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Testing the Raw model's configuration
messages = [
    {"role": "user", "content": "¿Cuál es el capital de Australia?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

output_ids = model.generate(input_ids, max_new_tokens=512)
response = tokenizer.batch_decode(output_ids)[0]
print(response)

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",
    target_parameters=[
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
        "23.mlp.experts.gate_up_proj",
        "23.mlp.experts.down_proj",
    ],
)
peft_model = get_peft_model(model, peft_config)
peft_model.config.use_cache = False  # Silence warnings, cache is incompatible with checkpointing
peft_model.print_trainable_parameters()

In [ ]:
from trl import SFTConfig

training_args = SFTConfig(
    learning_rate=2e-4,
    optim="adamw_torch",
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    max_length=1024,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    output_dir="gpt-oss-20b-BioRAG",
    report_to="trackio",
    push_to_hub=True,
)

In [ ]:
!nvidia-smi

#### Key points for the error 
* The reason of the above error is `quantization_config = Mxfp4Config(dequantize=True)` line of code. It is upcasting meaning it is changing it's precision from MXFP4 to bffloat16.
* The step is necessary for stable gradient accumulation and accuracy for better learning according to this [Fine-Tuning gpt-oss for Accuracy and Performance with Quantization Aware Training](https://developer.nvidia.com/blog/fine-tuning-gpt-oss-for-accuracy-and-performance-with-quantization-aware-training/)